In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import shutil
import time

session = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = "https://www.javbus.com/actresses/"

namelist = []
for page in range(1, 209):
    req = session.get(url+str(page), headers=headers)
    bsObj = BeautifulSoup(req.text)
    namelist.extend([{"title":i["title"], "src":i["src"]} for i in bsObj.findAll("img", {"src":re.compile(".*\.jpg")})])
    time.sleep(0.3)

for name in namelist:         
    response = requests.get(name["src"], stream=True, headers=headers)
    with open("database/"+name["title"]+'.jpg', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    time.sleep(0.3)

In [ ]:
import os
import face_recognition
import pprint
import sys

avtest = []
for root, dirs, files in os.walk("database"):
    for filename in files:
        if 'jpg' in filename:
            avtest.append("database/" + filename)

class target:
    def __init__(self, f):
        try:
            self.name = f
            self._img = face_recognition.load_image_file(f)
            self.encoding = face_recognition.face_encodings(self._img)[0]
            self.failed = False
        except:
            self.failed = True

extracted = [target(f) for f in avtest if target(f).failed == False]
all_encoding = [f.encoding for f in extracted]
all_name = [f.name for f in extracted]
t_img = face_recognition.load_image_file(sys.argv[1])
t_encoding = face_recognition.face_encodings(t_img)[0]


result = sorted(zip(face_recognition.api.face_distance(all_encoding, t_encoding), all_name))

pprint.pprint(result[0:5])